In [24]:
import os
import pandas as pd
import io

In [60]:
def create_coral_file_df(filename):
    coral_file = open(filename,'r')
    file_in = coral_file.readlines()

    for line in file_in:
        if "Location:" in line:
            location = line.split(": ")[1]
        if "Northernmost_Latitude:" in line:
            northernmost_lat = line.split(": ")[1]
        if "Southernmost_Latitude:" in line:
            southernmost_lat = line.split(": ")[1]
        if "Easternmost_Longitude:" in line:
            easternmost_lat = line.split(": ")[1]
        if "Westernmost_Longitude:" in line:
            westernmost_lat = line.split(": ")[1]
        if "Elevation:" in line:
            elevation = line.split(": ")[1].split(' ')[0]
        if "Earliest_Year:" in line:
            earliest_year = line.split(": ")[1]
        if "Most_Recent_Year:" in line:
            most_recent_year = line.split(": ")[1]
        if "Species_Name:" in line:
            species = line.split(": ")[1].strip('\n')
        if line.startswith('age'):
            age_line = i
            extracted = ' '.join(file_in[age_line:])

    lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
    lon = (float(easternmost_lat) + float(westernmost_lat)) / 2

    data = io.StringIO(extracted)
    df = pd.read_csv(data, sep="\t")
    df = df[['age', 'd18O']]
    df = df[df['d18O'] != 'NAN']
    df['lat'] = lat
    df['lon'] = lon
    df['elevation'] = elevation
    df['species'] = species
    print(df_test_d18O)
        

Quick Proof of Concept for the function above:

In [61]:
filename = 'swart1998-st-1_noaa.txt'
create_coral_file_df(filename)

     age   d18O   lat   lon elevation            species
2   1977  -3.60  0.47  6.58        -4  Siderastrea spp. 
3   1978  -3.48  0.47  6.58        -4  Siderastrea spp. 
4   1979  -3.59  0.47  6.58        -4  Siderastrea spp. 
5   1980  -3.69  0.47  6.58        -4  Siderastrea spp. 
6   1981  -3.50  0.47  6.58        -4  Siderastrea spp. 
7   1982  -3.52  0.47  6.58        -4  Siderastrea spp. 
8   1983  -3.66  0.47  6.58        -4  Siderastrea spp. 
9   1984  -3.85  0.47  6.58        -4  Siderastrea spp. 
10  1985  -3.81  0.47  6.58        -4  Siderastrea spp. 
11  1986  -3.51  0.47  6.58        -4  Siderastrea spp. 
12  1987  -3.48  0.47  6.58        -4  Siderastrea spp. 
13  1988  -3.27  0.47  6.58        -4  Siderastrea spp. 
14  1989  -3.80  0.47  6.58        -4  Siderastrea spp. 
15  1990  -3.68  0.47  6.58        -4  Siderastrea spp. 
16  1991  -3.28  0.47  6.58        -4  Siderastrea spp. 
17  1992  -3.63  0.47  6.58        -4  Siderastrea spp. 
18  1993  -3.51  0.47  6.58    

In [14]:
coral_file = 'swart1998-st-1_noaa.txt'
# Example of file with -noaa extension
file1 = open(coral_file,'r')
print(file1.read())
file1.close()

﻿# Gulf of Guinea Coral Isotope, Luminescence and Density Data
#----------------------------------------------------------------------- 
#                World Data Center for Paleoclimatology, Boulder 
#                                  and 
#                     NOAA Paleoclimatology Program 
#----------------------------------------------------------------------- 
# NOTE: Please cite Publication, and Online_Resource and date accessed when using these data. 
# If there is no publication information, please cite Investigators, Title, and Online_Resource and date accessed. 
#
# Online_Resource: https://www.ncdc.noaa.gov/study/1913
#
# Original_Source_URL: http://www1.ncdc.noaa.gov/pub/data/paleo/coral/atlantic/saotome1998.txt
# 
# Description/Documentation lines begin with #
# Data lines have no #
#
# Archive: Corals and Sclerosponges
#--------------------
# Contribution_Date
#	Date: 2005-02-01
#--------------------
# Title
#	Study_Name: Ponta Banana, Principe, Sao Tome - Isotope, Lumi

Gather metadata info! (site lat/lon & years of study)

In [58]:
file1 = open(coral_file,'r')
file_in = file1.readlines()

for line in file_in:
    if "Location:" in line:
        location = line.split(": ")[1]
    if "Northernmost_Latitude:" in line:
        northernmost_lat = line.split(": ")[1]
    if "Southernmost_Latitude:" in line:
        southernmost_lat = line.split(": ")[1]
    if "Easternmost_Longitude:" in line:
        easternmost_lat = line.split(": ")[1]
    if "Westernmost_Longitude:" in line:
        westernmost_lat = line.split(": ")[1]
    if "Elevation:" in line:
        elevation = line.split(": ")[1].split(' ')[0]
    if "Earliest_Year:" in line:
        earliest_year = line.split(": ")[1]
    if "Most_Recent_Year:" in line:
        most_recent_year = line.split(": ")[1]
    if "Species_Name:" in line:
        species = line.split(": ")[1].strip('\n')

lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
lon = (float(easternmost_lat) + float(westernmost_lat)) / 2



In [49]:
age_line = 0
file1 = open(coral_file,'r')
with file1 as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

91
age	d18O	d13C	lumin	dens
 1975	NAN	NAN	-13.274	NAN
 1976	NAN	NAN	1.392	1.664
 1977	-3.60	-1.05	4.481	1.646
 1978	-3.48	-1.17	5.966	1.645
 1979	-3.59	-1.07	7.250	1.686
 1980	-3.69	-1.03	15.983	1.590
 1981	-3.50	-0.87	5.236	1.712
 1982	-3.52	-1.15	10.030	1.675
 1983	-3.66	-1.23	13.752	1.560
 1984	-3.85	-0.89	9.509	1.472
 1985	-3.81	-1.10	5.770	1.483
 1986	-3.51	-1.01	6.514	1.744
 1987	-3.48	-1.12	-1.673	1.712
 1988	-3.27	-0.99	0.708	1.672
 1989	-3.80	-1.39	2.578	1.628
 1990	-3.68	-1.34	0.703	1.727
 1991	-3.28	-1.31	6.420	1.716
 1992	-3.63	-1.04	9.449	1.687
 1993	-3.51	-1.30	NAN	NAN



In [54]:
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

,age,d18O,d13C,lumin,dens
0,1975,NAN,NAN,-13.274,NAN
1,1976,NAN,NAN,1.392,1.664
2,1977,-3.60,-1.05,4.481,1.646
3,1978,-3.48,-1.17,5.966,1.645
4,1979,-3.59,-1.07,7.250,1.686


Just grab `age` and `d18O` columns! And remove NAN values. 

In [59]:
df_test_d18O = df_test[['age', 'd18O']]
df_test_d18O = df_test_d18O[df_test_d18O['d18O'] != 'NAN']
df_test_d18O['lat'] = lat
df_test_d18O['lon'] = lon
df_test_d18O['elevation'] = elevation
df_test_d18O['species'] = species
df_test_d18O

,age,d18O,lat,lon,elevation,species
2,1977,-3.60,0.47,6.58,-4,Siderastrea spp.
3,1978,-3.48,0.47,6.58,-4,Siderastrea spp.
4,1979,-3.59,0.47,6.58,-4,Siderastrea spp.
5,1980,-3.69,0.47,6.58,-4,Siderastrea spp.
6,1981,-3.50,0.47,6.58,-4,Siderastrea spp.
7,1982,-3.52,0.47,6.58,-4,Siderastrea spp.
8,1983,-3.66,0.47,6.58,-4,Siderastrea spp.
9,1984,-3.85,0.47,6.58,-4,Siderastrea spp.
10,1985,-3.81,0.47,6.58,-4,Siderastrea spp.
11,1986,-3.51,0.47,6.58,-4,Siderastrea spp.
